## Librerias

In [1]:
from ultralytics import YOLO
import cv2
import numpy as np
import time
import os
from collections import deque
from moviepy.editor import VideoFileClip, concatenate_videoclips

## Funcion para detectar si el balon toca el aro

In [2]:
def boxes_touch(box1, box2):
    x1_min, y1_min, x1_max, y1_max = box1
    x2_min, y2_min, x2_max, y2_max = box2
    return x1_min <= x2_max and x1_max >= x2_min and y1_min <= y2_max and y2_max >= y1_min

## Funcion para detectar si el jugador esta cerca del aro

In [3]:
def is_inside_circle(center, radius, box):
    x_center, y_center = center
    x1, y1, x2, y2 = box
    points_to_check = [(x1, y1), (x2, y1), (x1, y2), (x2, y2)]
    for (x, y) in points_to_check:
        if np.sqrt((x - x_center)**2 + (y - y_center)**2) <= radius:
            return True
    return False

## Funcion para realizar el clip de la jugada destacada

In [4]:
def save_clip(start_frames, end_frames, width, height, filename, fps):
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(filename, fourcc, fps, (width, height))
    for frame in start_frames + end_frames:
        out.write(frame)
    out.release()

## Funcion para calcular la distancia Jugador-Aro

In [5]:
def calculate_distance(point1, point2):
    return np.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

## Funcion para unir los clips

In [6]:
def concatenate_clips(clip_filenames, output_filename):
    clips = [VideoFileClip(filename) for filename in clip_filenames]
    final_clip = concatenate_videoclips(clips)
    final_clip.write_videofile(output_filename, codec="libx264")

## Procesamiento del video

In [7]:
def highlight(video_path):
    
    #Se crea la carpte IMG para almacenar las jugadas destacadas
    if not os.path.exists('IMG'):
        os.makedirs('IMG')

    #Se carga el modelo
    model = YOLO('Modelo25Pro.pt')
    """
    Este modelo posee 4 clases
    -0 Que corresponde al tablero
    -1 Que corresponde al balon
    -2 Que corresponde al aro
    -3 Que corresponde a los jugadores
    
    Se detectaran unicamente las clases 1, 2 y 3
    """
    model.predict(classes=[1, 2, 3])

    #Se entrega la ruta del video
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    #Se definen los parametros para posteriormente crear el clip de la jugada destacada
    buffer_seconds = 3
    buffer_size = int(fps * buffer_seconds)
    frame_buffer = deque(maxlen=buffer_size)
    
    #Se crean variables para la correcta creacion del clip de la jugada destacada
    recording = False
    end_buffer = []
    frames_after_touch = int(fps * 3)

    #Se crean variables para las futuras condiciones de jugadas destacadas
    last_player_center = None
    ball_touched_nothing = False
    frames_to_save_launch = None
    recording_launch = False
    clip_filenames = []

    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break

        start_time = time.perf_counter()
        results = model(frame, verbose=False)
        end_time = time.perf_counter()
        inference_time = end_time - start_time
        annotated_frame = results[0].plot()
        
        #Se crean variables para almacenar las coordenadas de los balones detectados, de los aros detectados, y de los jugadores detectados
        ball_coords, rim_coords, player_coords = [], [], []

        #Se guardan las coordenadas de cada clase detectada en su respectiva lista
        for result in results:
            for box in result.boxes:
                cls = int(box.cls.cpu().numpy()[0])
                coords = box.xyxy.cpu().numpy()[0]
                if cls == 1:
                    ball_coords.append(coords)
                elif cls == 2:
                    rim_coords.append(coords)
                elif cls == 3:
                    player_coords.append(coords)

        #Se obtienen las coordenadas del aro para proponer un area de deteccion sobre este
        if rim_coords:
            x_center = int((rim_coords[0][0] + rim_coords[0][2]) / 2)
            y_center = int((rim_coords[0][1] + rim_coords[0][3]) / 2)
            radius = 100
            cv2.circle(annotated_frame, (x_center, y_center), radius, (0, 255, 0), 2)

        #Se muestran los FPS
        cv2.putText(annotated_frame, f"FPS: {int(1 / inference_time)}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        cv2.imshow('Frame', annotated_frame)
        
        #Se toma el buffer antes creado con el frame actual
        frame_buffer.append(frame)


        #Condiciones para hacer clip de las jugadas destacadas
        
        #Condicion para un Dunk/Bandeja
        for ball_box in ball_coords:
            for rim_box in rim_coords:
                if boxes_touch(ball_box, rim_box):
                    for player_box in player_coords:
                        if is_inside_circle((x_center, y_center), radius, player_box):
                            if not recording:
                                recording = True
                                end_buffer = []
                                frames_to_save = list(frame_buffer)
                            video_time = int(cap.get(cv2.CAP_PROP_POS_MSEC) / 1000)
                            filename = f'IMG/{video_time}_Dunk_Bandeja.jpg'
                            cv2.imwrite(filename, annotated_frame)
                            break

        #Condicion para un triple
        for ball_box in ball_coords:
            if last_player_center is None:
                for player_box in player_coords:
                    if boxes_touch(ball_box, player_box):
                        last_player_center = ((player_box[0] + player_box[2]) // 2, (player_box[1] + player_box[3]) // 2)
                        ball_touched_nothing = True
                        break
            else:
                ball_touched_player = any(boxes_touch(ball_box, player_box) for player_box in player_coords)
                if not ball_touched_player:
                    if ball_touched_nothing:
                        for rim_box in rim_coords:
                            if boxes_touch(ball_box, rim_box):
                                distance = calculate_distance(last_player_center, (x_center, y_center))
                                if distance > 400:
                                    recording_launch = True
                                    end_buffer_launch = []
                                    frames_to_save_launch = list(frame_buffer)
                                    video_time = int(cap.get(cv2.CAP_PROP_POS_MSEC) / 1000)
                                    filename = f'IMG/{video_time}_Triple.jpg'
                                    cv2.imwrite(filename, annotated_frame)
                                last_player_center = None
                                ball_touched_nothing = False
                                break
                else:
                    last_player_center = ((player_box[0] + player_box[2]) // 2, (player_box[1] + player_box[3]) // 2)
                    ball_touched_nothing = True

        #Se guarda el clip del Dunk/Bandeja
        if recording:
            end_buffer.append(frame)
            if len(end_buffer) >= frames_after_touch:
                recording = False
                clip_filename = f'IMG/Clip_{video_time}.mp4'
                save_clip(frames_to_save, end_buffer, int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)), clip_filename, fps)
                clip_filenames.append(clip_filename)
                
        #Se guarda el clip del triple
        if recording_launch:
            end_buffer_launch.append(frame)
            if len(end_buffer_launch) >= frames_after_touch:
                recording_launch = False
                clip_filename_launch = f'IMG/Clip_{video_time}.mp4'
                save_clip(frames_to_save_launch, end_buffer_launch, int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)), clip_filename_launch, fps)
                clip_filenames.append(clip_filename_launch)
                
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()
    
    #Se unen los clips generados para obtener un video final
    if clip_filenames:
        concatenate_clips(clip_filenames, 'IMG/Highlights.mp4')

## Ejecucion del proyecto

In [8]:
video_path = "video2.mp4"
highlight(video_path)

WARNING  'source' is missing. Using 'source=C:\Users\Varfyo\Desktop\PC_2021\USM\U_11vo_semestre\IPD-441 Visin Por Computador\Proyecto\my-venv\Lib\site-packages\ultralytics\assets'.

image 1/2 C:\Users\Varfyo\Desktop\PC_2021\USM\U_11vo_semestre\IPD-441 Visin Por Computador\Proyecto\my-venv\Lib\site-packages\ultralytics\assets\bus.jpg: 640x480 3 Players, 119.7ms
image 2/2 C:\Users\Varfyo\Desktop\PC_2021\USM\U_11vo_semestre\IPD-441 Visin Por Computador\Proyecto\my-venv\Lib\site-packages\ultralytics\assets\zidane.jpg: 384x640 2 Players, 164.6ms
Speed: 12.0ms preprocess, 142.1ms inference, 200.1ms postprocess per image at shape (1, 3, 384, 640)
Moviepy - Building video IMG/Highlights.mp4.
Moviepy - Writing video IMG/Highlights.mp4



Moviepy - Done !
Moviepy - video ready IMG/Highlights.mp4
